In [1]:
# allows editing aoc_utils "live" without restarting kernel
# see https://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
# and https://stackoverflow.com/a/17551284
%load_ext autoreload
%autoreload 2

# Add the aoc_utils path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import aoc_utils
get_input = aoc_utils.get_input
print = aoc_utils.debug_print

timer = aoc_utils.start_timer()

In [2]:
# Useful imports
import re
from collections import defaultdict, deque
import heapq
import functools
import queue
import itertools
import math
import random

In [3]:
# aliases from utils
getnums = aoc_utils.getnums

In [4]:
def Day1(data=get_input(1, 2021)):
  nums = list(map(int, data))
  def p1():
    return len([i for i in range(1, len(nums)) if nums[i] > nums[i-1]])
  def p2():
    sums = [sum(nums[i:i+3]) for i in range(len(nums) - 2)]
    return len([i for i in range(1, len(sums)) if sums[i] > sums[i-1]])
    
  return p1(),p2()

assert Day1() == (1548, 1589)

In [5]:
def Day2(data=get_input(2,2021)):
  def p1():
    h,d=0,0
    for line in data:
      n = getnums(line)[0]
      if 'forward' in line:
        h += n
      elif 'down' in line:
        d += n
      else:
        d -= n
    return d * h
  def p2():
    h,d,aim=0,0,0
    for line in data:
      n = getnums(line)[0]
      if 'forward' in line:
        h += n
        d += aim * n
      elif 'down' in line:
        aim += n
      else:
        aim -= n
    return d * h
  return p1(),p2()

assert Day2() == (1694130, 1698850445),Day2()

In [47]:
from collections import Counter

def Day3(data = get_input(3,2021)):
  b2d = aoc_utils.bin_to_decimal

  def p1():
    cnts = [Counter() for _ in range(12)]
    for line in data:
      for idx,c in enumerate(line):
        cnts[idx].update(c)
    a = b2d(''.join([cnt.most_common()[0][0] for cnt in cnts]))
    b = b2d(''.join([cnt.most_common()[-1][0] for cnt in cnts]))
    return a*b

  def p2():
    def winnow(arr, most_com=True):
      for idx in range(len(arr[0])):
        cnt = Counter()
        for l in arr:
          cnt.update(l[idx])
        v = None
        if most_com:
          v = '0' if cnt['0'] > cnt['1'] else '1'
        else:
          v = '1' if cnt['1'] < cnt['0'] else '0'
        arr = [l for l in arr if l[idx] == v]
        if len(arr) == 1:
          return arr[0]

    O = winnow(data, most_com=True)
    C  = winnow(data, most_com=False)
    return b2d(O)*b2d(C)
  return p1(),p2()

assert Day3() == (1540244, 4203981)